# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayacucho,5.6639,-67.6236,30.28,67,38,1.32,VE,1724947387
1,1,alofi,-19.0595,-169.9187,18.94,94,20,2.57,NU,1724947388
2,2,adamstown,-25.0660,-130.1015,18.94,71,8,4.54,PN,1724947390
3,3,muli,2.9167,73.5667,28.40,75,16,4.08,MV,1724947391
4,4,lazaro cardenas,17.9583,-102.2000,29.31,70,70,0.75,MX,1724947287


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] < 27) & 
                                (city_data_df["Max Temp"] > 21) & 
                                (city_data_df["Wind Speed"] < 4.5) &
                                (city_data_df["Cloudiness"] == 0)].copy()

# Drop any rows with null values in place
ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,tsiombe,-25.3000,45.4833,23.66,59,0,3.11,MG,1724947537
173,173,gobabis,-22.4500,18.9667,25.89,11,0,2.63,NaN,1724947582
248,248,jining,35.4050,116.5814,23.18,87,0,2.01,CN,1724947666
333,333,dezhou,37.4486,116.2925,23.29,80,0,2.30,CN,1724947763
394,394,zhosaly,45.4878,64.0781,24.79,23,0,3.94,KZ,1724947754
461,461,carloforte,39.1450,8.3058,25.71,63,0,1.78,IT,1724947908
475,475,laguna,38.4210,-121.4238,21.47,70,0,2.57,US,1724947925
490,490,newman,37.3138,-121.0208,25.36,32,0,1.49,US,1724947943
494,494,mariental,-24.6333,17.9667,22.50,14,0,3.67,NaN,1724947947
563,563,ventspils,57.3894,21.5606,25.96,57,0,2.06,LV,1724948029


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayacucho,VE,5.6639,-67.6236,67,
1,alofi,NU,-19.0595,-169.9187,94,
2,adamstown,PN,-25.0660,-130.1015,71,
3,muli,MV,2.9167,73.5667,75,
4,lazaro cardenas,MX,17.9583,-102.2000,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
puerto ayacucho - nearest hotel: Hotel VENETUR Amazonas
alofi - nearest hotel: Matavai Resort
adamstown - nearest hotel: No hotel found
muli - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
bredasdorp - nearest hotel: Victoria Hotel
mitu - nearest hotel: Hotel Brasilia Vaupés
west island - nearest hotel: Cocos Village Bungalows
belyy yar - nearest hotel: No hotel found
altay - nearest hotel: Sunshine Holiday Hotel
albany - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
dazuocun - nearest hotel: No hotel found
umba - nearest hotel: Гостиница
anshun - nearest hotel: 希亚华瑞酒店
mikun' - nearest hotel: Саквояж
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
edinburgh of the seven seas - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
nemuro - nearest hotel: お宿エクハシ
shimoda - nearest hotel: ホテルウラガ
la glacerie - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto ayacucho,VE,5.6639,-67.6236,67,Hotel VENETUR Amazonas
1,alofi,NU,-19.0595,-169.9187,94,Matavai Resort
2,adamstown,PN,-25.0660,-130.1015,71,No hotel found
3,muli,MV,2.9167,73.5667,75,No hotel found
4,lazaro cardenas,MX,17.9583,-102.2000,70,Hotel Sol del Pacífico


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 1,
    size = "Humidity",
    color = "City",
    alpha=0.75,
    title="Humidity Map by City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)